<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/Final_PATE_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project : Final PATE analysis Project

So, we are basically going to first use teacher models which will be used for training the models and then we will use that model to predict the labels on our dataset , annotating the dataset.
Then, we will evaluate the Predictions by adding noise and running DP query and measuring epsilon-delta leakage.

In [0]:
# Imports here
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
import torch
from torch import nn
from torch import optim
import torchvision
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import torch.utils.data as data_utils
from PIL import Image, ImageFile
import json
from torch.optim import lr_scheduler
import time
import os
import argparse
import copy
import pandas as pd

In [0]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.MNIST(root="./data", download=True, train=True, transform=transform)

# Download and load the test data
testset = datasets.MNIST(root="./data", download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=1)

In [18]:
from torch.utils.data.sampler import SubsetRandomSampler

def split_train_set():
    train_len = len(trainset)
    indices = list(range(train_len))
    split = int(np.floor(0.01 * train_len))
    indice = []  #true indices to be fed to the PATE analysis algorithm
    for image,label in testloader:
      label = np.array(label)
      indice.append(label)
    indice = np.array(indice)
    indice = indice.flatten()
    print(indice.shape)
    trainloaders=[]
    for i in range(100):
        train_idx = indices[(i)*split:(i+1)*split] #splitting training set into 10 equal parts, each a seperate teacher
        train_sampler = SubsetRandomSampler(train_idx)
        trainloader = torch.utils.data.DataLoader(trainset,batch_size=16,sampler=train_sampler)
        trainloaders.append(trainloader)
    
    return trainloaders,indice #return teacher models and the true indices

teachers,indice = split_train_set()
print(len(teachers[0]))
num_teachers = 100
num_examples = 10000  #10,000 unlabelled datapoints in the testset
num_labels = 10

(10000,)
38


In [19]:
print(f"training examples contain : {len(train_data)}")
print(f"testing examples contain : {len(test_data)}")


print(len(testloader))

training examples contain : 60000
testing examples contain : 10000
10000


In [20]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [21]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #using the CUDA cores of Nvidia's Tesla K80
from torch import optim
model = nn.Sequential(nn.Dropout(p=0.2), #setting a dropout with a probability of 0.2 to counter overfitting
                     nn.Linear(784,512), #since the images are 28x28, we are giving a flattened input of 784 pixels
                     nn.ReLU(),
                     nn.Linear(512,256),
                     nn.ReLU(),
                     nn.Linear(256,10),
                     nn.LogSoftmax(dim=1)) #using a Log-Softmax activation for the output layer

criterion = nn.CrossEntropyLoss()  #calculating the cross entropy loss function
optimizer = optim.Adam(model.parameters(),lr=0.001) #using an Adam optimizer with a learning rate of 0.001

#Initialise 2 lists to later properly arrange the predictions made by all the teacher models
preds=[[]*10000]*100
temp_preds = []



model.to(device) #move the model to the GPU

for i in range(num_teachers):
    run_loss,val_loss_gr = [],[]
    epochs = 10  #train each teacher model for 10 epochs
    print(f"Training for teacher:{i+1}")
    for e in range(epochs):
        running_loss = 0
        val_loss = 0
        for images, labels in teachers[i]:
            images = images.view(images.shape[0],-1)
            images, labels = images.to(device),labels.to(device)
            output = model(images)
            optimizer.zero_grad()  #setting gradients to 0 before every epoch
            loss = criterion(output,labels)
            loss.backward() #our favourite back propagation
            optimizer.step()
            
            running_loss+=loss.item()
            
        else:
            with torch.no_grad(): #switch off gradient calculation for predictions
                model.eval() #disable dropout while making predictions
                temp = []  #list for storing predictions of all data points
                for image,label in testloader:
                    image = image.view(image.shape[0],-1)
                    image,label = image.to(device),label.to(device)
                    output = model(image)
                    loss = criterion(output,label)
                    ps = torch.exp(output) #finding the probablity distribution for an image
                    top_p,top_class = ps.topk(1,dim=1) #find the class that the model predicted
                    equals = top_class == label.view(*top_class.shape)
                    top_class = top_class.cpu()  #converting a GPU tensor to a normal CPU tensor for further processing
                    temp.append(top_class.flatten())
                    val_loss+=loss.item()
            print(f'Accuracy: {torch.sum(equals.type(torch.FloatTensor))}')
            print(f"Loss:{running_loss/len(teachers[i])} epoch:{e+1}")
            print(f"Val_Loss:{val_loss/len(testloader)} epoch:{e+1}")
        model.train() #re-enable dropout


    temp = np.array(temp)
    print(temp.shape)
    preds[i-1] = temp
    

preds = np.array(preds)
print(preds.shape)#predictions made by all 10 teacher sets stored in this list



            


Training for teacher:1
Accuracy: 0.0
Loss:1.6191779845639278 epoch:1
Val_Loss:0.9497288597663311 epoch:1
Accuracy: 1.0
Loss:0.6508175064074365 epoch:2
Val_Loss:0.6400350536797385 epoch:2
Accuracy: 1.0
Loss:0.4211721065405168 epoch:3
Val_Loss:0.7294591390825129 epoch:3
Accuracy: 1.0
Loss:0.3647241250268723 epoch:4
Val_Loss:0.7590557443005604 epoch:4
Accuracy: 1.0
Loss:0.33355933349383504 epoch:5
Val_Loss:0.519907944195293 epoch:5
Accuracy: 1.0
Loss:0.24145698890482126 epoch:6
Val_Loss:0.5532780552995294 epoch:6
Accuracy: 1.0
Loss:0.2139308770236216 epoch:7
Val_Loss:0.4768547105686333 epoch:7
Accuracy: 1.0
Loss:0.2563328174384017 epoch:8
Val_Loss:0.5761154664102718 epoch:8
Accuracy: 1.0
Loss:0.15938007373264745 epoch:9
Val_Loss:0.6320510263193848 epoch:9
Accuracy: 1.0
Loss:0.17078137922247774 epoch:10
Val_Loss:0.7699149685981024 epoch:10
(10000,)
Training for teacher:2
Accuracy: 1.0
Loss:0.892522320151329 epoch:1
Val_Loss:0.5070327971476306 epoch:1
Accuracy: 1.0
Loss:0.5166293935556161 e

In [0]:
pip install syft

With noise in the actual labels and applying PATE analysis


In [24]:
#!pip install syft  #installing the PySyft framework which has the PATE analysis API
from syft.frameworks.torch.differential_privacy import pate

pred = preds.transpose()
new_labels = list()

for an_image in pred:
  label_counts = np.bincount(an_image,minlength=10)
  epsilon = 0.1
  beta = 1/epsilon
  for i in range(len(label_counts)):
    label_counts[i]+= np.random.laplace(0,beta,1)  #adding Laplacian noise to every true indice to hide sensitive information
  
  new_label = np.argmax(label_counts)
  new_labels.append(new_label)
  
new_labels = np.array(new_labels)


  
print(new_labels.shape)
print(pred.shape)


data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=pred.transpose(), indices=new_labels, noise_eps=0.1, delta=1e-5) #calculating the epsilon scores using PATE Analysis
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

W0814 16:50:23.807709 140442849232768 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0814 16:50:23.820248 140442849232768 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



(10000,)
(10000, 100)
Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 56.37146069554869


We will be adding Laplacian noise to the predicted labels.



In [25]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=pred.transpose(), indices=indice, noise_eps=0.1, delta=1e-5) #calculating the epsilon scores using PATE Analysis
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 56.53886728152238
